In [1]:
import os
import re
import csv
import codecs
import pandas as pd

In [2]:
def to_date(**kwargs):
    """
    Funcao que recebe o ano (YYYY), ano e mes (YYYYmm) ou ano, mes e dia (YYYYmmdd)
    no tipo string e o transforma no tipo date.
    
    Args:
        date_reference(str): Ano, ano e mes ou ano, mes e dia.
            Ex.: '2011' ou '201105' ou '20110526'
    
    Returns:
        date: Data de referencia.
            Ex.: '2011-05-26'
    """
    
    # Define o ano, mes e dia de referencia do arquivo
    if len(kwargs['date_reference']) == 4:
        year = kwargs['date_reference'][:4]
        month = '01'
        day = '01'
    elif len(kwargs['date_reference']) == 6:
        year = kwargs['date_reference'][:4]
        month = kwargs['date_reference'][4:6]
        day = '01'
    elif len(kwargs['date_reference']) == 8:
        year = kwargs['date_reference'][:4]
        month = kwargs['date_reference'][4:6]
        day = kwargs['date_reference'][6:]
    
    # Define a data de referencia do arquivo
    date = pd.to_datetime(year+'-'+month+'-'+day)
    
    return date

In [3]:
def file_references(**kwargs):
    """
    Funcao que recebe o nome do arquivo original e extrai o assunto,
    data e estado de referencia.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '20180420_Quadro_AC.csv'
    
    Returns:
        date: Data de referencia do arquivo.
            Ex.: '2018-04-20'
        string: Assunto de referencia do arquivo.
            Ex.: 'Quadro'
        string: Estado de referencia do arquivo.
            Ex.: 'AC'
    """
    # Expressao Regular para:
    # Identificar a data de referencia no nome do arquivo
    date_reference_re = re.compile(r'^[0-9]+')
    # Identificar o assunto no nome do arquivo
    file_subject_re = re.compile(r'_([\w]+)_([\w]+)\.')
    
    # Identifica a data de referencia do arquivo
    kwargs['date_reference'] = re.search(date_reference_re, kwargs['data_file']).group()
        
    # Define a data de referencia do arquivo
    date = to_date(**kwargs)
    
    # Define o assunto do arquivo
    subject = re.search(file_subject_re, kwargs['data_file']).group(1)
    
    # Define o estado de referencia do arquivo
    state = re.search(file_subject_re, kwargs['data_file']).group(2)
    
    return date, subject, state
    

In [4]:
def secondary_data_path(secondary_name, **kwargs):
    """
    Funcao que monta o nome dos arquivos secundarios gerados durante a
    padronizacao do arquivo original.
    """
    return kwargs['out_data_path'].replace(kwargs['data_file'], secondary_name + '_' + kwargs['data_file'])

In [5]:
def clean_string(text):
    """
    Funcao que recebe um texto (string) e o padroniza.
    Os passos são:
    1. Insere o termo SIGILOSO nos registros protegidos por sigilo.
    2. Insere o termo INDISPONIVEL nos registros com detalhamento nao disponivel.
    3. Remove espacos brancos extras no final dos registros.
    
    Args:
        text(str): Texto de um registro do arquivo original.
            Ex.: 'Texto com espacos extras    '
    
    Returns:
        str: Texto padronizado.
            Ex.: 'Texto com espacos extras'
    """
    # Expressao regular que identifica campos protegidos por sigilo
    confidential_re = re.compile(r'protegidas por sigilo')
    unavailable_re = re.compile(r'Detalhamento das informa')
    
    if pd.isnull(text):
        return text
    else:
        c = confidential_re.search(text)
        i = unavailable_re.search(text)
        if c:
            return u'SIGILOSO'
        elif i:
            return u'INDISPONIVEL'
        else:
            return text.strip()

In [6]:
def describe_cd_qualificacao(text):
    """
    Funcao que recebe o codigo da qualificacao do socio e retorna
    a sua descricao.
    
    Args:
        text(str): Codigo da qualificacao do socio.
            Ex.: '54'
    
    Returns:
        str: Descricao da qualificacao do socio.
            Ex.: 'Fundador'
    """
    # Dominio de qualificacao do socio
    domain = {'05': 'Administrador'
             ,'08': 'Conselheiro de Administração'
             ,'10': 'Diretor'
             ,'16': 'Presidente'
             ,'17': 'Procurador'
             ,'20': 'Sociedade Consorciada'
             ,'21': 'Sociedade Filiada'
             ,'22': 'Sócio'
             ,'23': 'Sócio Capitalista'
             ,'24': 'Sócio Comanditado'
             ,'25': 'Sócio Comanditário'
             ,'26': 'Sócio de Indústria'
             ,'28': 'Sócio-Gerente'
             ,'29': 'Sócio Incapaz ou Relat.Incapaz (exceto menor)'
             ,'30': 'Sócio Menor (Assistido/Representado)'
             ,'31': 'Sócio Ostensivo'
             ,'37': 'Sócio Pessoa Jurídica Domiciliado no Exterior'
             ,'38': 'Sócio Pessoa Física Residente no Exterior'
             ,'47': 'Sócio Pessoa Física Residente no Brasil'
             ,'48': 'Sócio Pessoa Jurídica Domiciliado no Brasil'
             ,'49': 'Sócio-Administrador'
             ,'52': 'Sócio com Capital'
             ,'53': 'Sócio sem Capital'
             ,'54': 'Fundador'
             ,'55': 'Sócio Comanditado Residente no Exterior'
             ,'56': 'Sócio Comanditário Pessoa Física Residente no Exterior'
             ,'57': 'Sócio Comanditário Pessoa Jurídica Domiciliado no Exterior'
             ,'58': 'Sócio Comanditário Incapaz'
             ,'59': 'Produtor Rural'
             ,'63': 'Cotas em Tesouraria'
             ,'65': 'Titular Pessoa Física Residente ou Domiciliado no Brasil'
             ,'66': 'Titular Pessoa Física Residente ou Domiciliado no Exterior'
             ,'67': 'Titular Pessoa Física Incapaz ou Relativamente Incapaz (exceto menor)'
             ,'68': 'Titular Pessoa Física Menor (Assistido/Representado)'
             ,'70': 'Administrador Residente ou Domiciliado no Exterior'
             ,'71': 'Conselheiro de Administração Residente ou Domiciliado no Exterior'
             ,'72': 'Diretor Residente ou Domiciliado no Exterior'
             ,'73': 'Presidente Residente ou Domiciliado no Exterior'
             ,'74': 'Sócio-Administrador Residente ou Domiciliado no Exterior'
             ,'75': 'Fundador Residente ou Domiciliado no Exterior'}
    
    if text in domain.keys():
        return domain[text]
    else:
        return 'Descrição do código não informada.'

In [7]:
def describe_in_cpf_cnpj(text):
    """
    Funcao que recebe o codigo do indicador de CPF/CNPJ
    e retorna sua descricao.
    
    Args:
        text(str): Codigo do indicador de CPF/CNPJ.
            Ex.: '2'
    
    Returns:
        str: Descricao do indicador de CPF/CNPJ.
            Ex.: 'Pessoa Fisica'
    """
    # Dominio de qualificacao do socio
    domain = {'1': 'Pessoa Jurídica'
             ,'2': 'Pessoa Física'
             ,'3': 'Nome Exterior'}
    
    if text in domain.keys():
        return domain[text]
    else:
        return 'Descrição do código não informada.'

In [8]:
def load_data_frame(**kwargs):
    """
    Funcao que recebe o caminho completo de um arquivo de dados ja codificado
    em UTF-8, prepara-o em um data frame e renomeia as colunas.
    
    Args:
        in_fields(array): Lista com os nomes das colunas para o data frame.
            Ex.: ['cd_campo', 'nm_campo', 'dt_campo', 'vl_campo']
        in_data_path(str): Caminho completo de acesso ao arquivo de dados.
            Ex.: '..\\data\\encoded\\201301_GastosDiretos.csv'
        
    Returns:
        dataframe: Data frame do arquivo original com as colunas renomeadas.
    """
    # Le o arquivo original em um dataframe
    df = pd.read_csv(kwargs['in_data_path']
                    ,sep='\t'
                    ,quoting = csv.QUOTE_NONE
                    ,low_memory = True
                    ,encoding = 'utf-8'
                    ,dtype = 'object')
    
    #Renomeia as colunas
    df.columns = kwargs['in_fields']
    
    return df

In [9]:
def data_frame_to_csv(df, out_data_path):
    """
    Funcao que recebe o caminho completo para escrita do dataframe e o
    transforma em arquivo CSV codificado em UTF-8.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        out_data_path(str): Caminho completo para escrita do dataframe em csv.
            Ex.: '..\\data\\padronized\\Favorecidos_GastosDiretos_2013-01-01.csv'
    """
    df.to_csv(path_or_buf = out_data_path
             ,index = False
             ,sep = '\t'
             ,encoding = 'utf-8')

In [10]:
def group_distinct(df, fields_list):
    """
    Funcao que recebe um dataframe e agrupa os diferentes valores das
    colunas informadas.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        fields_list(array): Lista de colunas que serao agrupadas
            em valores distintos.
            Ex.: ['cd_campo', 'nm_campo', 'cd_subcampo', 'nm_subcampo']
    
    Returns:
        dataframe: Dataframe com os diferentes valores das colunas informadas.
    """
    return df.loc[:,fields_list].drop_duplicates().copy()

In [11]:
def select_fields(df, fields_list):
    """
    Funcao que recebe um data frame e retorna uma copia com apenas as
    colunas informadas.
    
    Args:
        df(dataframe): Data frame enviado para recorte.
        fields_list(array): Lista de colunas do dataframe para selecao.
        
    Returns:
        dataframe: Copia do dataframe apenas com as colunas informadas.
    """
    return df.loc[:,fields_list].copy()

In [12]:
def add_data_source_fields(df, **kwargs):
    """
    Funcao que adiciona ao final do dataframe uma sequencia de campos
    referentes a fonte dos dados que o originou.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2013-01-01'
        data_source(str): Nome da fonte do arquivo original.
            Ex.: 'Portal da Transparência'
        data_file(str): Nome do arquivo original.
            Ex,: 'GastosDiretos_201301.csv'
        
    Returns:
        dataframe: Dataframe com os campos referentes a 
            fonte de dados adicionado.
    """
    df['dt_referencia'] = kwargs['date_reference']
    df['nm_fonte_dados'] = kwargs['data_source']
    df['nm_arquivo_dados'] = kwargs['data_file']
    return df

In [13]:
def create_branch_files(df, **kwargs):
    """
    Funcao que divide o dataframe padronizado em arquivos secundarios. Cada
    arquivo e criado conforme o dicionario de dados enviado, sendo a chave (key)
    o nome do arquivo secundario e os valores (value) a lista de campos que 
    devem constar no arquivo. 
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        files_dict(dict): Dicionario de dados com o nome dos arquivos secundarios
            (key) e lista de campos para o arquivo (value).
            Ex.: {'Favorecido': ['cd_favorecido', 'nm_favorecido']}
        out_data_path(str): Caminho completo para escrita do arquivo secundario.
        
    """
    # Nome das colunas adicionadas em cada arquivo
    data_source_fields = ['dt_referencia'
                         ,'nm_fonte_dados'
                         ,'nm_arquivo_dados']
    
    # Define data e assunto de referência do arquivo
    # date_reference, subject, _ = file_references(file_csv)
    
    for key, value in kwargs['out_files'].iteritems():
        value.extend(data_source_fields)
        
        if key == kwargs['subject']:
            sub_df = select_fields(df, value)
        else:
            sub_df = group_distinct(df, value)
            
        sub_dp = secondary_data_path(key, **kwargs)
        data_frame_to_csv(sub_df, sub_dp)
        print 'Arquivo ' + sub_dp + ' pronto'
    

In [14]:
def wrangle_cnpj(**kwargs):
    """
    Recebe a localização fisica do arquivo original de CPNJ, padroniza
    os dados do arquivo e cria um novo arquivo no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '20130101_CNPJ_ES.csv'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2013-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\organized\20130101_CNPJ_ES.csv'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\wrangled\20130101_CNPJ_ES.csv'
    """
    
    # Nome das colunas que irão substituir o nome das colunas originais
    kwargs['in_fields'] = ['tp_campo'
                          ,'nr_cnpj'
                          ,'nm_razao_social']

    # Prepara um dicionario para receber os campos
    # dos arquivos principal e secundarios
    kwargs['out_files'] = dict()
    
    # Definicao dos campos do arquivo principal
    # Nome das colunas do arquivo de gastos diretos
    kwargs['out_files']['CNPJ'] = ['tp_campo'
                                  ,'nr_cnpj'
                                  ,'nm_razao_social'
                                  ,'sg_uf']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(**kwargs)
    
    # Padroniza os dados do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Inclui o campo de estado
    df['sg_uf'] = kwargs['state']

    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Separa o arquivo original em outros conforme definido no dicionario de arquivos de saida
    create_branch_files(df, **kwargs)

    print 'Arquivo padronizado.'

In [15]:
def wrangle_quadro(**kwargs):
    """
    Recebe a localização fisica do arquivo original de Gastos Diretos, padroniza
    os dados do arquivo e cria  novo arquivo no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '201301_GastosDiretos.csv'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2013-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\encoded\201301_GastosDiretos.csv'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\padronized\201301_GastosDiretos.csv'
    """
    
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['in_fields'] = ['tp_campo'
                          ,'nr_cnpj'
                          ,'in_cpf_cnpj'
                          ,'nr_cpf_cnpj_socio'
                          ,'cd_qualificacao_socio'
                          ,'nm_socio']
    
    # Prepara um dicionario para receber os campos
    # dos arquivos principal e secundarios
    kwargs['out_files'] = dict()
    
    # Definicao dos campos do arquivo principal
    # Nome das colunas do arquivo de socios
    kwargs['out_files']['Quadro'] = ['tp_campo'
                                    ,'nr_cnpj'
                                    ,'in_cpf_cnpj'
                                    ,'ds_cpf_cnpj'
                                    ,'nr_cpf_cnpj_socio'
                                    ,'cd_qualificacao_socio'
                                    ,'ds_qualificacao_socio'
                                    ,'nm_socio']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(**kwargs)
    
    # Padroniza todos os campos do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Descreve os campos com codigo
    df['ds_qualificacao_socio'] = df['cd_qualificacao_socio'].apply(describe_cd_qualificacao)
    df['ds_cpf_cnpj'] = df['in_cpf_cnpj'].apply(describe_in_cpf_cnpj)
    
    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Separa o arquivo original em outros conforme definido no dicionario de arquivos de saida
    create_branch_files(df, **kwargs)
    
    print 'Arquivo padronizado.'      

In [16]:
def wrangle_files():
    """
    Funcao responsavel por transformar e padronizar os dados do arquivo de entrada, e
    dividi-los em blocos de fatos e dimensoes.
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = dict()
    
    # Diretório onde são armazenados os arquivos originais
    kwargs['in_data_dir'] = '..\\..\\data\\03-organized'
    kwargs['out_data_dir'] = '..\\..\\data\\04-wrangled'
    
    # Nome da fonte de dados
    kwargs['data_source'] = u'Receita Federal'
    
    # Lista dos arquivos originais 
    files = os.listdir(kwargs['in_data_dir'])

    # Para cada arquivo na lista de arquivos originais
    for data_file in files:
        
        # Define o nome do arquivo no dicionario
        kwargs['data_file'] = data_file

        # Define data e assunto de referência do arquivo
        kwargs['date_reference'], \
        kwargs['subject'], \
        kwargs['state'] = file_references(**kwargs)

        # Define o caminho completo de acesso ao arquivo original
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])

        # Define o caminho completo de armazenamento do arquivo de saida
        kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['data_file'])

        if kwargs['subject'] == 'CNPJ':
            wrangle_cnpj(**kwargs)
        elif kwargs['subject'] == 'Quadro':
            wrangle_quadro(**kwargs)
        

In [ ]:
def main():
    ### Transforma os arquivos
    wrangle_files()

if __name__ == '__main__':
    main()

Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_AC.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_AL.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_AM.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_AP.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_BA.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_CE.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_DF.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_ES.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_GO.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_MA.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_MG.csv pronto
Arquivo padronizado.
Arquivo ..\..\data\04-wrangled\CNPJ_20180718_CNPJ_MS.csv pronto
A